In [ ]:
s2c[Theta_, Phi_] := {Sin[Theta]*Cos[Phi], Sin[Theta]*Sin[Phi], Cos[Theta]}
s2ctangent[Theta_, Phi_] := {Sin[Phi], -Cos[Phi], 0.0}
s2cbitangent[Theta_, Phi_] := Cross[s2ctangent[Theta, Phi], s2c[Theta, Phi]]
Lerp[a_, b_, t_] := a + (b - a)*t
Saturate[u_] := Min[1.0, Max[0.0, u]]

In [ ]:
(* GGX - the final BRDF is multiplied by the cosine from the rendering equation - we refer to that as CBRDF in the notes!*)
SchlickFresnel[u_] := Power[Saturate[1 - u], 5.0]
GTR2[NdotH_, a_] := (a^2)/(Pi*(1 + (a^2 - 1)*NdotH*NdotH)^2)
SmithG[NdotV_, alphaG_] := 2.0/( 1 + Sqrt[1 + alphaG*alphaG*(1 - NdotV*NdotV)/(NdotV*NdotV)])
H[ThetaLight_, PhiLight_, ThetaView_] := Normalize[s2c[ThetaLight, PhiLight] + s2c[ThetaView, 0.0]]
NdotH[ThetaLight_, PhiLight_, ThetaView_] := H[ThetaLight, PhiLight, ThetaView][[3]]
LdotH[ThetaLight_, PhiLight_, ThetaView_] := H[ThetaLight, PhiLight, ThetaView].s2c[ThetaLight, PhiLight]
RoughnessRemap[roughness_] := (roughness*0.5 + 0.5)^2
BRDF[ThetaLight_, PhiLight_, ThetaView_, roughness_, C0spec_] := 
    SmithG[Cos[ThetaLight], RoughnessRemap[roughness]] * SmithG[Cos[ThetaView], RoughnessRemap[roughness]] *
    Lerp[C0spec, 1, SchlickFresnel[LdotH[ThetaLight, PhiLight, ThetaView]]] *
    GTR2[NdotH[ThetaLight, PhiLight, ThetaView], roughness^2] /
    (4*Cos[ThetaLight]*Cos[ThetaView]) * 
    Cos[ThetaLight]

In [ ]:
(* our approximate model *)
ApproxModelN[i_, numLobes_] := ( i + 1)*2 - ( numLobes + 1)
ApproxModelLobeTheta[i_, numLobes_, Theta_, spread_] := Theta + (i/(numLobes - 1)*2 - 1) * spread
ApproxModelD[i_, numLobes_, Theta_, spread_] :=  ApproxModelN[i, numLobes]/2*(ApproxModelLobeTheta[i, numLobes, Theta, spread] - Theta)/spread
ApproxModelLobeWeight[i_, numLobes_, Theta_, spread_] := Exp[ -(spread/3) * ApproxModelD[i, numLobes, Theta, spread]^2]
ApproxModelLobe[i_, numLobes_, Theta_, baseAmplitude_, Lambda_,  spread_,  Thetalight_, PhiLight_] := 
    baseAmplitude * ApproxModelLobeWeight[i, numLobes, Theta, spread]*
    Exp[-Lambda*(1 - Dot[s2c[ApproxModelLobeTheta[i, numLobes, Theta, spread], Pi], s2c[Thetalight, PhiLight]])]
ApproxModel[ numLobes_, Theta_, baseAmplitude_, Lambda_, spread_, Thetalight_, PhiLight_] := Sum[ApproxModelLobe[i, numLobes,  Theta, baseAmplitude, Lambda, spread,  Thetalight, PhiLight], {i, 0, numLobes - 1}]

In [ ]:
(* fit data for the approximate mode - amplitude set so that integral of the approx model matches the integral of the GGX lobe *)
ApproxModelFitDataEqualEnergy = {{{0.00120, 334.46265, 20976.32422, 
     0.00139}, {0.36583, 348.88553, 22026.46484, 0.00000}, {0.52140, 
     346.83844, 22026.46484, 0.00000}, {0.63977, 344.32016, 
     22026.46484, 0.00000}, {0.74603, 343.49554, 22026.46484, 
     0.00508}, {0.83735, 337.34402, 22026.46484, 0.00000}, {0.92945, 
     332.49969, 22026.46484, 0.00000}, {1.00738, 326.31296, 
     22026.46484, 0.00000}, {1.08440, 318.15640, 22026.46484, 
     0.00000}, {1.15841, 306.90167, 22026.46484, 0.00000}, {1.23449, 
     290.47739, 22026.46484, 0.00000}, {1.30302, 265.12219, 
     22026.46484, 0.00000}, {1.36857, 224.37569, 22026.46484, 
     0.00000}, {1.44228, 159.28596, 22026.46484, 0.00000}, {1.50617, 
     67.47960, 22026.46484, 0.00000}, {1.57080, 0.26174, 22026.46484, 
     0.00000}}, {{0.00000, 325.03064, 20422.99414, 0.00000}, {0.36721,
      349.52289, 22026.46484, 0.00230}, {0.52231, 346.39041, 
     22026.46484, 0.00102}, {0.64350, 342.80948, 22026.46484, 
     0.00000}, {0.74757, 338.96555, 22026.46484, 0.00000}, {0.84106, 
     334.19287, 22026.46484, 0.00000}, {0.92728, 328.20987, 
     22026.46484, 0.00000}, {1.00824, 320.57907, 22026.46484, 
     0.00000}, {1.08525, 310.57993, 22026.46484, 0.00000}, {1.15923, 
     296.98468, 22026.46484, 0.00000}, {1.23089, 277.67044, 
     22026.46484, 0.00000}, {1.30075, 249.06456, 22026.46484, 
     0.00000}, {1.36926, 205.65427, 22026.46484, 0.00000}, {1.43671, 
     141.20126, 22026.46484, 0.00000}, {1.50322, 57.85246, 
     22026.46484, 0.00000}, {1.54999, 1.66747, 22026.46484, 
     0.01717}}, {{0.00000, 20.35447, 1278.28662, 0.00088}, {0.36719, 
     23.10183, 1439.04211, 0.01301}, {0.52226, 26.17008, 1635.06946, 
     0.01797}, {0.64342, 29.82670, 1876.88025, 0.02143}, {0.74746, 
     34.25863, 2179.84863, 0.02404}, {0.84090, 39.71511, 2566.77075, 
     0.02604}, {0.92707, 46.55170, 3072.98169, 0.02759}, {1.00794, 
     55.30333, 3756.62085, 0.02877}, {1.08484, 66.82079, 4720.56543, 
     0.02968}, {1.15865, 82.53018, 6162.92627, 0.03041}, {1.23004, 
     104.97863, 8509.97461, 0.03106}, {1.29943, 138.83575, 
     12799.68262, 0.03176}, {1.36703, 192.16679, 21991.24609, 
     0.03263}, {1.43230, 128.95212, 22026.46484, 0.03048}, {1.49181, 
     56.27706, 22026.46484, 0.03050}, {1.49627, 10.31521, 22026.46484,
      0.05944}}, {{0.00000, 4.02004, 253.46632, 0.00000}, {0.36708, 
     4.65602, 285.45822, 0.02951}, {0.52205, 5.30914, 324.74057, 
     0.04115}, {0.64307, 6.07417, 373.35620, 0.04935}, {0.74693, 
     6.99139, 434.47076, 0.05557}, {0.84014, 8.11056, 512.82544, 
     0.06040}, {0.92600, 9.50092, 615.81702, 0.06417}, {1.00647, 
     11.26576, 755.76501, 0.06710}, {1.08281, 13.56586, 954.30627, 
     0.06940}, {1.15580, 16.66325, 1252.80078, 0.07126}, {1.22590, 
     20.99740, 1737.69470, 0.07292}, {1.29314, 27.26119, 2605.54419, 
     0.07462}, {1.35680, 36.22188, 4341.82373, 0.07655}, {1.41416, 
     46.15429, 8043.11816, 0.07849}, {1.45578, 37.14472, 12246.42480, 
     0.07896}, {1.43678, 7.84672, 6103.37695, 0.11388}}, {{0.00000, 
     1.27640, 81.09336, 0.00000}, {0.36674, 1.51599, 91.31808, 
     0.05262}, {0.52137, 1.74626, 104.11358, 0.07472}, {0.64196, 
     2.01006, 119.99376, 0.09038}, {0.74527, 2.32175, 140.00226, 
     0.10235}, {0.83779, 2.69745, 165.72742, 0.11173}, {0.92274, 
     3.15859, 199.62343, 0.11910}, {1.00200, 3.73543, 245.69957, 
     0.12486}, {1.07667, 4.47243, 310.80405, 0.12935}, {1.14726, 
     5.43375, 407.24963, 0.13288}, {1.21378, 6.70316, 557.95465, 
     0.13573}, {1.27542, 8.34546, 805.75623, 0.13815}, {1.32999, 
     10.21687, 1221.19177, 0.14044}, {1.37229, 11.33813, 1843.98413, 
     0.14389}, {1.39068, 8.95659, 2224.56250, 0.15544}, {1.36140, 
     3.54222, 1391.89087, 0.20611}}, {{0.00000, 0.52667, 34.00076, 
     0.00000}, {0.36582, 0.64350, 38.24704, 0.08118}, {0.51961, 
     0.75167, 43.73625, 0.11891}, {0.63915, 0.87173, 50.54110, 
     0.14561}, {0.74115, 1.01050, 59.09336, 0.16607}, {0.83198, 
     1.17423, 70.03307, 0.18204}, {0.91477, 1.37015, 84.30383, 
     0.19446}, {0.99120, 1.60713, 103.35500, 0.20393}, {1.06207, 
     1.89553, 129.42630, 0.21090}, {1.12752, 2.24413, 165.93362, 
     0.21571}, {1.18698, 2.65178, 217.87305, 0.21875}, {1.23894, 
     3.08146, 290.87909, 0.22060}, {1.28069, 3.41280, 385.81573, 
     0.22332}, {1.30734, 3.39191, 480.74084, 0.23206}, {1.31020, 
     2.71702, 500.88126, 0.25951}, {1.27718, 1.53757, 368.16672, 
     0.31776}}, {{0.00000, 0.25719, 17.08466, 0.00000}, {0.36362, 
     0.32251, 19.15609, 0.11169}, {0.51554, 0.38325, 21.96175, 
     0.17194}, {0.63282, 0.44746, 25.40183, 0.21381}, {0.73201, 
     0.51877, 29.65478, 0.24528}, {0.81936, 0.59947, 34.97277, 
     0.26920}, {0.89781, 0.69135, 41.69038, 0.28710}, {0.96882, 
     0.79553, 50.24834, 0.29995}, {1.03301, 0.91183, 61.20561, 
     0.30859}, {1.09018, 1.03670, 75.17523, 0.31385}, {1.13981, 
     1.16038, 92.57336, 0.31740}, {1.18043, 1.26335, 113.10121, 
     0.32176}, {1.20965, 1.31060, 134.35741, 0.33084}, {1.22381, 
     1.24715, 149.11081, 0.34869}, {1.21693, 1.03357, 145.43858, 
     0.38515}, {1.18341, 0.70509, 116.22989, 0.44006}}, {{0.00000, 
     0.14158, 9.83939, 0.00000}, {0.35862, 0.18032, 10.95893, 
     0.13687}, {0.50666, 0.21800, 12.55153, 0.22615}, {0.61946, 
     0.25485, 14.45112, 0.28554}, {0.71345, 0.29340, 16.72048, 
     0.32903}, {0.79470, 0.33415, 19.43098, 0.36070}, {0.86613, 
     0.37768, 22.68630, 0.38414}, {0.92908, 0.42305, 26.55891, 
     0.40058}, {0.98437, 0.46865, 31.09266, 0.41235}, {1.03182, 
     0.51241, 36.31380, 0.42141}, {1.07100, 0.55028, 42.07780, 
     0.43068}, {1.10080, 0.57510, 47.89766, 0.44279}, {1.11926, 
     0.57647, 52.73867, 0.45958}, {1.12427, 0.54260, 54.87890, 
     0.48453}, {1.11151, 0.47000, 52.57357, 0.52523}, {1.07822, 
     0.36430, 44.83702, 0.57740}}, {{0.00000, 0.08555, 6.34574, 
     0.00000}, {0.34772, 0.10886, 6.99428, 0.14877}, {0.48857, 
     0.13277, 7.93296, 0.26644}, {0.59388, 0.15386, 9.00241, 
     0.34117}, {0.67989, 0.17479, 10.23308, 0.39701}, {0.75275, 
     0.19566, 11.63044, 0.43918}, {0.81514, 0.21557, 13.17577, 
     0.46811}, {0.86866, 0.23624, 14.93886, 0.49505}, {0.91404, 
     0.25502, 16.82965, 0.51472}, {0.95131, 0.27095, 18.78398, 
     0.53133}, {0.98063, 0.28287, 20.69666, 0.54873}, {1.00117, 
     0.28942, 22.40899, 0.57081}, {1.01110, 0.28715, 23.59692, 
     0.59660}, {1.00935, 0.27356, 23.87062, 0.63008}, {0.99292, 
     0.24737, 22.90496, 0.67264}, {0.96112, 0.20850, 20.42159, 
     0.71686}}, {{0.00000, 0.05591, 4.52487, 0.00000}, {0.32669, 
     0.07166, 4.93114, 0.16813}, {0.45605, 0.08541, 5.47000, 
     0.28854}, {0.55086, 0.09748, 6.06776, 0.37533}, {0.62658, 
     0.10861, 6.72406, 0.43927}, {0.68921, 0.11898, 7.41958, 
     0.49001}, {0.74163, 0.12863, 8.15174, 0.53076}, {0.78545, 
     0.13770, 8.92124, 0.56544}, {0.82133, 0.14662, 9.73157, 
     0.60166}, {0.84953, 0.15359, 10.49350, 0.63190}, {0.86988, 
     0.15835, 11.17195, 0.65871}, {0.88244, 0.16133, 11.73278, 
     0.69445}, {0.88532, 0.15959, 11.99811, 0.72475}, {0.87930, 
     0.15524, 12.03221, 0.76395}, {0.86077, 0.14541, 11.60883, 
     0.80613}, {0.82997, 0.13149, 10.77899, 0.84817}}, {{0.00001, 
     0.03887, 3.50888, 0.00001}, {0.29215, 0.05352, 3.81872, 
     0.25000}, {0.40545, 0.06063, 4.12603, 0.34877}, {0.48717, 
     0.06636, 4.43709, 0.41730}, {0.55087, 0.07202, 4.76815, 
     0.48158}, {0.60273, 0.07736, 5.11207, 0.53714}, {0.64503, 
     0.08192, 5.44628, 0.58073}, {0.67886, 0.08680, 5.80459, 
     0.62848}, {0.70559, 0.09073, 6.11875, 0.67287}, {0.72552, 
     0.09428, 6.42424, 0.71383}, {0.73985, 0.09625, 6.66088, 
     0.73985}, {0.74902, 0.09510, 6.71609, 0.74902}, {0.75117, 
     0.09277, 6.69209, 0.75117}, {0.74769, 0.08848, 6.51979, 
     0.74769}, {0.73436, 0.08221, 6.19544, 0.73436}, {0.71131, 
     0.07449, 5.74070, 0.71131}}, {{0.00000, 0.02828, 2.90502, 
     0.00000}, {0.24654, 0.03818, 3.08968, 0.24654}, {0.33909, 
     0.04239, 3.25904, 0.33909}, {0.40576, 0.04570, 3.42379, 
     0.40576}, {0.45636, 0.04831, 3.57172, 0.45636}, {0.49921, 
     0.05063, 3.71400, 0.49921}, {0.53172, 0.05244, 3.83821, 
     0.53172}, {0.55833, 0.05410, 3.96016, 0.55833}, {0.58032, 
     0.05534, 4.05622, 0.58032}, {0.59631, 0.05614, 4.12652, 
     0.59631}, {0.60499, 0.05631, 4.15620, 0.60499}, {0.60980, 
     0.05612, 4.15790, 0.60980}, {0.60986, 0.05535, 4.11451, 
     0.60986}, {0.60420, 0.05419, 4.04058, 0.60420}, {0.58746, 
     0.05227, 3.91543, 0.58746}, {0.56805, 0.05006, 3.76079, 
     0.56805}}, {{0.00000, 0.02118, 2.52203, 0.00000}, {0.18909, 
     0.02685, 2.61618, 0.18909}, {0.26285, 0.02922, 2.69299, 
     0.26285}, {0.31194, 0.03097, 2.76533, 0.31194}, {0.35103, 
     0.03238, 2.82491, 0.35103}, {0.38117, 0.03366, 2.88589, 
     0.38117}, {0.40604, 0.03467, 2.92909, 0.40604}, {0.42401, 
     0.03553, 2.96700, 0.42401}, {0.43890, 0.03634, 2.99916, 
     0.43890}, {0.44893, 0.03695, 3.01715, 0.44893}, {0.45549, 
     0.03743, 3.02261, 0.45549}, {0.45698, 0.03776, 3.01571, 
     0.45698}, {0.45403, 0.03789, 2.99177, 0.45403}, {0.44801, 
     0.03807, 2.96768, 0.44801}, {0.43928, 0.03804, 2.92131, 
     0.43928}, {0.42324, 0.03784, 2.86764, 0.42324}}, {{0.00000, 
     0.01621, 2.27125, 0.00000}, {0.12775, 0.01921, 2.30619, 
     0.12775}, {0.17585, 0.02054, 2.33608, 0.17585}, {0.20963, 
     0.02162, 2.36525, 0.20963}, {0.23464, 0.02252, 2.38821, 
     0.23464}, {0.25397, 0.02329, 2.40273, 0.25397}, {0.26937, 
     0.02406, 2.42058, 0.26937}, {0.28282, 0.02480, 2.43221, 
     0.28282}, {0.28982, 0.02546, 2.44075, 0.28982}, {0.29634, 
     0.02613, 2.44469, 0.29634}, {0.29925, 0.02681, 2.44823, 
     0.29925}, {0.30035, 0.02743, 2.44093, 0.30035}, {0.29942, 
     0.02808, 2.43204, 0.29942}, {0.29590, 0.02877, 2.42196, 
     0.29590}, {0.28948, 0.02949, 2.40867, 0.28948}, {0.27929, 
     0.03016, 2.39305, 0.27929}}, {{0.00036, 0.01259, 2.10020, 
     0.00036}, {0.06161, 0.01392, 2.10953, 0.06161}, {0.08458, 
     0.01461, 2.11068, 0.08458}, {0.10031, 0.01525, 2.11812, 
     0.10031}, {0.11309, 0.01591, 2.12742, 0.11309}, {0.12260, 
     0.01651, 2.13110, 0.12260}, {0.12929, 0.01710, 2.13168, 
     0.12929}, {0.13463, 0.01773, 2.13567, 0.13463}, {0.13974, 
     0.01840, 2.13702, 0.13974}, {0.14298, 0.01908, 2.13529, 
     0.14298}, {0.14511, 0.01979, 2.13224, 0.14511}, {0.14511, 
     0.02057, 2.13224, 0.14511}, {0.14511, 0.02145, 2.13224, 
     0.14511}, {0.14380, 0.02235, 2.12721, 0.14380}, {0.14183, 
     0.02337, 2.12235, 0.14183}, {0.13902, 0.02445, 2.11852, 
     0.13902}}, {{0.00052, 0.00988, 1.98367, 0.00000}, {0.00000, 
     0.01020, 1.98044, 0.00000}, {0.00000, 0.01057, 1.98044, 
     0.00000}, {0.00000, 0.01096, 1.98044, 0.00000}, {0.00000, 
     0.01138, 1.98012, 0.00000}, {0.00000, 0.01184, 1.98012, 
     0.00000}, {0.00000, 0.01234, 1.98012, 0.00000}, {0.00000, 
     0.01288, 1.98012, 0.00000}, {0.00000, 0.01347, 1.98012, 
     0.00000}, {0.00000, 0.01410, 1.97758, 0.00000}, {0.00006, 
     0.01479, 1.97488, 0.00000}, {0.00000, 0.01558, 1.97473, 
     0.00000}, {0.00000, 0.01646, 1.97473, 0.00000}, {0.00000, 
     0.01745, 1.97473, 0.00000}, {0.00000, 0.01856, 1.97473, 
     0.00000}, {0.00390, 0.01981, 1.96873, 0.00390}}};


In [ ]:
(* fit data for the approximate model - amplitude is just a free variable *)
ApproxModelFitDataFreeAmplitude = {{{0.00173, 147.41316, 22026.46484, 
     0.00000}, {0.36347, 147.41316, 22026.46484, 0.00000}, {0.51857, 
     147.41316, 22026.46484, 0.00000}, {0.63977, 147.41316, 
     22026.46484, 0.00000}, {0.74385, 147.41316, 22026.46484, 
     0.00000}, {0.83735, 147.41316, 22026.46484, 0.00000}, {0.92358, 
     147.41316, 22026.46484, 0.00000}, {1.00738, 147.41316, 
     22026.46484, 0.00000}, {1.08427, 147.41316, 16572.91797, 
     0.00000}, {1.15556, 147.41316, 22026.46484, 0.00000}, {1.23449, 
     147.41316, 22026.46484, 0.00000}, {1.30000, 147.41316, 
     22026.46484, 0.00000}, {1.37369, 147.41316, 22026.46484, 
     0.00000}, {1.44135, 147.41316, 22026.46484, 0.00000}, {1.50739, 
     119.39033, 22026.46484, 0.00000}, {1.57080, 0.46573, 22026.46484,
      0.00000}}, {{0.00000, 147.41316, 12109.05273, 
     0.00000}, {0.36721, 147.41316, 12264.74414, 0.00000}, {0.52231, 
     147.41316, 12456.16016, 0.00000}, {0.64349, 147.41316, 
     12690.07910, 0.00000}, {0.74757, 147.41316, 12975.19531, 
     0.00000}, {0.84106, 147.41316, 13323.37891, 0.00000}, {0.92728, 
     147.41316, 13751.57031, 0.00000}, {1.00823, 147.41316, 
     14286.41797, 0.00000}, {1.08524, 147.41316, 14973.68457, 
     0.00000}, {1.15923, 147.41316, 15898.83008, 0.00000}, {1.23088, 
     147.41316, 17239.03516, 0.00000}, {1.30074, 147.41316, 
     19409.52734, 0.00000}, {1.36925, 147.41316, 22026.46484, 
     0.00000}, {1.43671, 147.41316, 22026.46484, 0.00000}, {1.50322, 
     72.48656, 22026.46484, 0.00210}, {1.55333, 1.29984, 22024.20117, 
     0.01187}}, {{0.00000, 21.67491, 1789.81067, 0.00062}, {0.36719, 
     24.78167, 2033.22070, 0.01184}, {0.52227, 28.28209, 2333.20044, 
     0.01622}, {0.64343, 32.46874, 2707.63574, 0.01920}, {0.74748, 
     37.55893, 3182.53711, 0.02136}, {0.84093, 43.84870, 3797.17798, 
     0.02296}, {0.92710, 51.76856, 4613.44629, 0.02414}, {1.00799, 
     61.98147, 5735.50879, 0.02498}, {1.08491, 75.56459, 7351.70508, 
     0.02558}, {1.15876, 94.34849, 9831.80273, 0.02601}, {1.23021, 
     121.57184, 13977.74707, 0.02634}, {1.29969, 147.41316, 
     19495.46875, 0.02624}, {1.36732, 147.41316, 22026.46484, 
     0.02681}, {1.43258, 113.79637, 22026.46484, 0.02789}, {1.49219, 
     52.76747, 22026.46484, 0.02915}, {1.50491, 7.15401, 22026.46484, 
     0.04790}}, {{0.00000, 4.27716, 353.09680, 0.00000}, {0.36710, 
     4.98001, 401.20822, 0.02693}, {0.52208, 5.71233, 460.62946, 
     0.03714}, {0.64313, 6.57407, 534.87842, 0.04413}, {0.74702, 
     7.61020, 629.19873, 0.04927}, {0.84027, 8.87889, 751.56006, 
     0.05311}, {0.92619, 10.46138, 914.49725, 0.05597}, {1.00674, 
     12.48174, 1139.24133, 0.05807}, {1.08320, 15.13723, 1464.04529, 
     0.05959}, {1.15637, 18.75109, 1963.44214, 0.06068}, {1.22677, 
     23.86570, 2796.34888, 0.06149}, {1.29454, 31.34964, 4339.45508, 
     0.06208}, {1.35931, 42.36621, 7621.88818, 0.06231}, {1.41935, 
     55.66885, 15627.01270, 0.06090}, {1.46596, 39.78098, 22026.46484,
      0.05838}, {1.47113, 12.32440, 22026.46484, 0.06448}}, {{0.00000,
      1.35355, 111.43874, 0.00000}, {0.36678, 1.61388, 126.61186, 
     0.04853}, {0.52148, 1.86521, 145.47629, 0.06761}, {0.64215, 
     2.15503, 169.10237, 0.08084}, {0.74558, 2.49890, 199.18146, 
     0.09065}, {0.83823, 2.91456, 238.22786, 0.09809}, {0.92338, 
     3.42691, 290.41690, 0.10366}, {1.00290, 4.07200, 362.54382, 
     0.10776}, {1.07794, 4.90419, 466.81790, 0.11067}, {1.14913, 
     6.00488, 626.19830, 0.11255}, {1.21661, 7.49435, 887.62909, 
     0.11349}, {1.27993, 9.51216, 1353.11877, 0.11334}, {1.33778, 
     12.11559, 2264.78198, 0.11163}, {1.38705, 14.74773, 4196.11865, 
     0.10738}, {1.42167, 14.08201, 7579.35156, 0.09877}, {1.42562, 
     8.34688, 10010.61230, 0.09494}}, {{0.00000, 0.55480, 45.44004, 
     0.00000}, {0.36598, 0.68156, 51.62596, 0.07713}, {0.51991, 
     0.79528, 59.41609, 0.10878}, {0.63966, 0.92310, 69.12220, 
     0.13113}, {0.74193, 1.07186, 81.53430, 0.14771}, {0.83312, 
     1.24879, 97.67577, 0.16029}, {0.91639, 1.46193, 119.18229, 
     0.16956}, {0.99351, 1.72384, 148.80243, 0.17610}, {1.06541, 
     2.04959, 191.24808, 0.18012}, {1.13234, 2.45715, 254.58810, 
     0.18175}, {1.19416, 2.96501, 354.00620, 0.18090}, {1.24994, 
     3.57532, 518.03998, 0.17746}, {1.29811, 4.24328, 801.75989, 
     0.17156}, {1.33581, 4.79615, 1297.41394, 0.16414}, {1.35855, 
     4.82675, 2082.61768, 0.15740}, {1.35740, 3.78996, 2750.06372, 
     0.15453}}, {{0.00000, 0.26819, 21.77683, 0.00056}, {0.36399, 
     0.33956, 24.72528, 0.11074}, {0.51620, 0.40088, 28.50568, 
     0.16053}, {0.63404, 0.46757, 33.24966, 0.19550}, {0.73376, 
     0.54239, 39.24471, 0.22097}, {0.82184, 0.62867, 47.00327, 
     0.23992}, {0.90148, 0.72923, 57.26583, 0.25309}, {0.97403, 
     0.84673, 71.16600, 0.26092}, {1.04022, 0.98325, 90.36525, 
     0.26446}, {1.10044, 1.14111, 117.76173, 0.26359}, {1.15420, 
     1.32163, 158.15411, 0.25939}, {1.20109, 1.51041, 217.71344, 
     0.25216}, {1.23943, 1.68991, 307.67853, 0.24340}, {1.26791, 
     1.82289, 439.47488, 0.23516}, {1.28256, 1.83626, 615.42206, 
     0.23040}, {1.27774, 1.63948, 774.37592, 0.23006}}, {{0.00000, 
     0.14614, 11.70444, 0.00403}, {0.36214, 0.20311, 13.64247, 
     0.19062}, {0.50788, 0.23011, 15.49789, 0.23079}, {0.62466, 
     0.26220, 17.59915, 0.26847}, {0.71694, 0.30470, 21.16785, 
     0.30600}, {0.79898, 0.34848, 25.18760, 0.33050}, {0.87309, 
     0.39454, 30.18275, 0.34513}, {0.93835, 0.44705, 37.05423, 
     0.35023}, {0.99799, 0.51462, 46.72785, 0.36222}, {1.04921, 
     0.57047, 58.10174, 0.35429}, {1.09394, 0.63649, 74.14941, 
     0.35004}, {1.13332, 0.69784, 96.00468, 0.34020}, {1.16313, 
     0.75630, 124.99036, 0.33475}, {1.18372, 0.79636, 162.10149, 
     0.32818}, {1.19203, 0.80090, 204.40877, 0.32592}, {1.18572, 
     0.75132, 240.80360, 0.32822}}, {{0.00087, 0.08647, 6.91903, 
     0.00261}, {0.34732, 0.09509, 7.34194, 0.05998}, {0.49023, 
     0.13640, 9.03917, 0.26832}, {0.59578, 0.17018, 10.90028, 
     0.39062}, {0.68390, 0.17516, 12.09146, 0.36067}, {0.75965, 
     0.20306, 14.38353, 0.41676}, {0.82483, 0.22659, 17.00770, 
     0.43823}, {0.88244, 0.25221, 20.60945, 0.44979}, {0.93173, 
     0.27429, 24.30886, 0.44991}, {0.97563, 0.30438, 29.74173, 
     0.45861}, {1.01349, 0.32907, 36.28755, 0.45215}, {1.04291, 
     0.35045, 43.58867, 0.45161}, {1.06689, 0.35982, 51.82888, 
     0.43671}, {1.07945, 0.37718, 61.83330, 0.44585}, {1.08106, 
     0.36721, 69.51354, 0.44256}, {1.06866, 0.35814, 78.75588, 
     0.45216}}, {{0.00009, 0.05594, 4.60760, 0.00000}, {0.32751, 
     0.07281, 5.12821, 0.18095}, {0.45767, 0.09307, 5.97727, 
     0.36309}, {0.55208, 0.10557, 6.82875, 0.43804}, {0.62862, 
     0.11495, 7.77301, 0.46609}, {0.70022, 0.12662, 8.91096, 
     0.49807}, {0.75714, 0.13184, 9.92918, 0.50290}, {0.79743, 
     0.15104, 11.72234, 0.56594}, {0.84299, 0.15871, 13.35382, 
     0.55743}, {0.87724, 0.16711, 15.21249, 0.55506}, {0.90756, 
     0.17450, 17.39609, 0.55387}, {0.92550, 0.18340, 19.65062, 
     0.56178}, {0.94388, 0.19456, 22.70360, 0.58499}, {0.95181, 
     0.19856, 25.26136, 0.59914}, {0.94424, 0.20144, 27.55205, 
     0.62322}, {0.92940, 0.18993, 28.40712, 0.62784}}, {{0.00000, 
     0.03892, 3.42600, 0.00000}, {0.29023, 0.05567, 3.82456, 
     0.29023}, {0.40442, 0.06376, 4.23122, 0.40442}, {0.48824, 
     0.06736, 4.59020, 0.42862}, {0.55236, 0.07431, 5.07775, 
     0.50847}, {0.60939, 0.07642, 5.49154, 0.50433}, {0.65414, 
     0.08061, 5.99069, 0.54213}, {0.69217, 0.08817, 6.73154, 
     0.59591}, {0.71827, 0.09659, 7.54945, 0.66588}, {0.74914, 
     0.09890, 8.05362, 0.67570}, {0.76515, 0.10462, 8.86091, 
     0.70965}, {0.78427, 0.10656, 9.42251, 0.71984}, {0.78766, 
     0.11171, 10.30548, 0.74796}, {0.78824, 0.11266, 10.79436, 
     0.77240}, {0.78046, 0.11016, 10.94466, 0.78046}, {0.76677, 
     0.10323, 10.61051, 0.76677}}, {{0.00044, 0.02842, 2.77520, 
     0.00044}, {0.24420, 0.03817, 2.97609, 0.24420}, {0.33659, 
     0.04222, 3.16500, 0.33659}, {0.40467, 0.04558, 3.37165, 
     0.40467}, {0.45681, 0.04829, 3.57181, 0.45681}, {0.50106, 
     0.05074, 3.77164, 0.50106}, {0.53546, 0.05301, 3.99439, 
     0.53546}, {0.56420, 0.05471, 4.17300, 0.56420}, {0.58708, 
     0.05629, 4.35561, 0.58708}, {0.60593, 0.05745, 4.51368, 
     0.60593}, {0.61789, 0.05839, 4.65313, 0.61789}, {0.62642, 
     0.05860, 4.74868, 0.62642}, {0.62827, 0.05844, 4.78193, 
     0.62827}, {0.62351, 0.05724, 4.71410, 0.62351}, {0.61194, 
     0.05550, 4.60186, 0.61194}, {0.58946, 0.05278, 4.35112, 
     0.58946}}, {{0.00000, 0.02138, 2.38912, 0.00000}, {0.18879, 
     0.02697, 2.48732, 0.18879}, {0.25960, 0.02937, 2.57968, 
     0.25960}, {0.30854, 0.03096, 2.66271, 0.30854}, {0.34887, 
     0.03243, 2.74305, 0.34887}, {0.37898, 0.03370, 2.81587, 
     0.37898}, {0.40045, 0.03451, 2.86692, 0.40045}, {0.42548, 
     0.03566, 2.94598, 0.42548}, {0.44051, 0.03638, 2.97684, 
     0.44051}, {0.44919, 0.03689, 2.99571, 0.44919}, {0.45650, 
     0.03747, 3.03298, 0.45650}, {0.45594, 0.03769, 3.02315, 
     0.45594}, {0.45788, 0.03795, 3.00900, 0.45788}, {0.44876, 
     0.03802, 2.98747, 0.44876}, {0.44045, 0.03793, 2.93361, 
     0.44045}, {0.42280, 0.03774, 2.87737, 0.42280}}, {{0.00002, 
     0.01645, 2.15537, 0.00002}, {0.12817, 0.01950, 2.18599, 
     0.12817}, {0.17177, 0.02067, 2.22190, 0.17177}, {0.20598, 
     0.02172, 2.24468, 0.20598}, {0.23180, 0.02265, 2.28370, 
     0.23180}, {0.24941, 0.02337, 2.29835, 0.24941}, {0.26220, 
     0.02409, 2.33211, 0.26220}, {0.27679, 0.02483, 2.33953, 
     0.27679}, {0.28707, 0.02556, 2.35345, 0.28707}, {0.29662, 
     0.02622, 2.35687, 0.29662}, {0.29616, 0.02685, 2.36264, 
     0.29616}, {0.29461, 0.02741, 2.34541, 0.29461}, {0.29447, 
     0.02813, 2.35035, 0.29447}, {0.29009, 0.02878, 2.33981, 
     0.29009}, {0.28845, 0.02954, 2.33606, 0.28845}, {0.27533, 
     0.03021, 2.31376, 0.27533}}, {{0.00370, 0.01289, 2.00171, 
     0.00370}, {0.06028, 0.01410, 2.00582, 0.06028}, {0.08543, 
     0.01498, 2.00008, 0.08543}, {0.09939, 0.01545, 2.01513, 
     0.09939}, {0.11107, 0.01608, 2.01853, 0.11107}, {0.12295, 
     0.01685, 2.03087, 0.12295}, {0.12701, 0.01724, 2.02944, 
     0.12701}, {0.13346, 0.01805, 2.04532, 0.13346}, {0.13799, 
     0.01857, 2.03504, 0.13799}, {0.14022, 0.01932, 2.03145, 
     0.14022}, {0.13875, 0.01996, 2.03575, 0.13875}, {0.14274, 
     0.02089, 2.04694, 0.14274}, {0.14459, 0.02172, 2.03781, 
     0.14459}, {0.13767, 0.02251, 2.02802, 0.13767}, {0.14217, 
     0.02377, 2.02715, 0.14217}, {0.13536, 0.02472, 2.02092, 
     0.13536}}, {{0.00000, 0.01003, 1.88565, 0.00000}, {0.00000, 
     0.01040, 1.88904, 0.00000}, {0.00000, 0.01074, 1.91695, 
     0.00000}, {0.00328, 0.01113, 1.87580, 0.00328}, {0.00250, 
     0.01160, 1.87404, 0.00250}, {0.00073, 0.01210, 1.90518, 
     0.00073}, {0.00014, 0.01261, 1.89925, 0.00014}, {0.00033, 
     0.01308, 1.86538, 0.00033}, {0.00165, 0.01370, 1.88418, 
     0.00165}, {0.00000, 0.01428, 1.86860, 0.00000}, {0.00023, 
     0.01507, 1.88240, 0.00023}, {0.00152, 0.01592, 1.88578, 
     0.00152}, {0.00362, 0.01691, 1.88981, 0.00362}, {0.00076, 
     0.01775, 1.87791, 0.00076}, {0.00699, 0.01904, 1.87704, 
     0.00699}, {0.00787, 0.02026, 1.87177, 0.00787}}};


In [ ]:
ThetaToFitIndex[dataSource_, Theta_] := 
 Floor[(1 - Cos[Theta])*(Length[dataSource[[1]]] - 1) + 0.5] + 1
RoughnessToFitIndex[dataSource_, roughness_] := 
 Floor[roughness*(Length[dataSource] - 1) + 0.5] + 1

ApproxModelFitGetTheta[dataSource_, Thetaview_, roughness_] := 
 dataSource[[RoughnessToFitIndex[dataSource, roughness]]][[
   ThetaToFitIndex[dataSource, Thetaview]]][[1]]
ApproxModelFitGetAmplitude[dataSource_, Thetaview_, roughness_] := 
 dataSource[[RoughnessToFitIndex[dataSource, roughness]]][[
   ThetaToFitIndex[dataSource, Thetaview]]][[2]]
ApproxModelFitGetLambda[dataSource_, Thetaview_, roughness_] := 
 dataSource[[RoughnessToFitIndex[dataSource, roughness]]][[
   ThetaToFitIndex[dataSource, Thetaview]]][[3]]
ApproxModelFitGetSpread[dataSource_, Thetaview_, roughness_] := 
 Max[0.00001, 
  dataSource[[RoughnessToFitIndex[dataSource, roughness]]][[
    ThetaToFitIndex[dataSource, Thetaview]]][[4]]]
ApproxModelFitLobeTheta[dataSource_, i_, Thetaview_, roughnes_] := 
 ApproxModelLobeTheta[i, 8, 
  ApproxModelFitGetTheta[dataSource, Thetaview, roughnes],  
  ApproxModelFitGetSpread[dataSource, Thetaview, roughnes]]

ApproxModelFitEqualEnergy[Thetalight_, Philight_, Thetaview_, roughness_] := 
 ApproxModel[8, 
  ApproxModelFitGetTheta[ApproxModelFitDataEqualEnergy, Thetaview, 
   roughness], 
                                                                  
  ApproxModelFitGetAmplitude[
   ApproxModelFitDataEqualEnergy, Thetaview, roughness],
                                                                  
  ApproxModelFitGetLambda[
   ApproxModelFitDataEqualEnergy, Thetaview, roughness],
                                                                  
  ApproxModelFitGetSpread[
   ApproxModelFitDataEqualEnergy, Thetaview, 
   roughness], Thetalight, Philight]

ApproxModelFitFreeAmplitude[Thetalight_, Philight_, Thetaview_, roughness_] := 
 ApproxModel[8, 
  ApproxModelFitGetTheta[
   ApproxModelFitDataFreeAmplitude, Thetaview, roughness], 
                                                                       
  ApproxModelFitGetAmplitude[
   ApproxModelFitDataFreeAmplitude, Thetaview, roughness],
                                                                       
  ApproxModelFitGetLambda[
   ApproxModelFitDataFreeAmplitude, Thetaview, roughness],
                                                                       
  ApproxModelFitGetSpread[
   ApproxModelFitDataFreeAmplitude, Thetaview, 
   roughness], Thetalight, Philight]


In [ ]:
(* some plots - GGX itself *)
Manipulate[ 
    Show[
        SphericalPlot3D[BRDF[\[Theta], \[Phi], \[Theta]view, roughness, C0spec], {\[Theta], 0, Pi/2}, {\[Phi], -Pi, Pi} , PlotRange -> 3, PlotStyle -> Directive[Green]]
    ], 
    {{\[Theta]view, 0.5, "View \[Theta]"}, 0, 0.5*Pi},
    {{roughness, 0.35, "Roughness" }, 0, 1},
    {{C0spec, 0.8, "C0"}, 0, 1}
 ]

In [ ]:
(* approximate model *)
LobeDirPlot[i_, numLobes_, \[Theta]_, spread_] := Graphics3D[{Thick, Blue, Line[{{0, 0, 0}, 7 *ApproxModelLobeWeight[i, numLobes, \[Theta], spread]* s2c[ApproxModelLobeTheta[i, numLobes, \[Theta], spread], Pi]}]}]

Manipulate[Show[SphericalPlot3D[ApproxModel[8, theta, ampl, lambda, spread, \[Theta], \[Phi]], {\[Theta], 0, Pi}, {\[Phi], -Pi, Pi} , PlotRange -> 10, PlotStyle -> Directive[Opacity[0.5]]],
                LobeDirPlot[0, 8, theta, spread],
                LobeDirPlot[1, 8, theta, spread],
                LobeDirPlot[2, 8, theta, spread],
                LobeDirPlot[3, 8, theta, spread],
                LobeDirPlot[4, 8, theta, spread],
                LobeDirPlot[5, 8, theta, spread],
                LobeDirPlot[6, 8, theta, spread],
                LobeDirPlot[7, 8, theta, spread]],
            {{theta, 0.5, "Model \[Theta]"}, 0, Pi/2},
            {{ampl, 1.0, "Model amplitude"}, 0, 3},
            {{lambda, 10.0, "Model \[Lambda]"}, 0, 20},
            {{spread, 0.2, "Model spread"}, 0.01, 3}
]

In [ ]:
(* GGX and both fit approximate models *)
Manipulate[ 
    Show[
         SphericalPlot3D[BRDF[\[Theta], \[Phi], \[Theta]view, roughness, 0.8], {\[Theta], 0, Pi/2}, {\[Phi], -Pi, Pi} , PlotRange -> 20, PlotStyle -> Directive[Orange, Opacity[0.3]]],
         SphericalPlot3D[ApproxModelFitEqualEnergy[\[Theta], \[Phi], \[Theta]view, roughness], {\[Theta], 0, Pi}, {\[Phi], -Pi, Pi} , PlotRange -> 20, PlotStyle -> Directive[Green, Opacity[0.3]]],
         SphericalPlot3D[ApproxModelFitFreeAmplitude[\[Theta], \[Phi], \[Theta]view, roughness], {\[Theta], 0, Pi}, {\[Phi], -Pi, Pi} , PlotRange -> 20, PlotStyle -> Directive[Blue, Opacity[0.3]]]
        ], 
    {{\[Theta]view, 0.5, "View \[Theta]"}, 0, 0.5*Pi},
    {{roughness, 0.35, "Roughness"}, 0.01, 1}
]

In [ ]:
(*plot the fit data - equal energy*)
ListPlot3D[Table[ApproxModelFitDataEqualEnergy[[i]][[j]][[1]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[Log[ApproxModelFitDataEqualEnergy[[i]][[j]][[2]]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[Log[ApproxModelFitDataEqualEnergy[[i]][[j]][[3]]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[ApproxModelFitDataEqualEnergy[[i]][[j]][[4]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]

In [ ]:
(*plot the fit data - free amplitude*)
ListPlot3D[Table[ApproxModelFitDataFreeAmplitude[[i]][[j]][[1]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[Log[ApproxModelFitDataFreeAmplitude[[i]][[j]][[2]]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[Log[ApproxModelFitDataFreeAmplitude[[i]][[j]][[3]]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]
ListPlot3D[Table[ApproxModelFitDataFreeAmplitude[[i]][[j]][[4]], {i, 1, 16}, {j, 1, 16}], DataRange -> {{0, 1}, {0, 1}}, AxesLabel -> {"1-N.V", "roughness"}, Boxed -> False]